# This notebook shows an example where a set of electrodes are selected from a dataset and then LFP is extracted from those electrodes and then written to a new NWB file


## The following steps must be completed before running this notebook:
### 1) Set up access to a DataJoint database. See https://tutorials.datajoint.io/
### 2) Install pynwb. 
    See https://pynwb.readthedocs.io/en/stable/getting_started.html#installation
### 3) Install the ndx-fl-novela package. 
     conda install -c novelakrk ndx-fl-novela
### 4) Download the example franklab NWB file from DropBox. 
    https://www.dropbox.com/to_be_updated

#### Make sure we're in the franklabnwb/franklabnwb directory. 
#### Eventually this will be unnecessary because the package will properly installed

#### Load all of the relevant modules and set the environment variables. 
Note that the datadir and datadir/analysis must exist

In [2]:
%env DJ_SUPPORT_FILEPATH_MANAGEMENT=TRUE
%load_ext autoreload
%autoreload 2


import pynwb
import os

#DataJoint and DataJoint schema
import datajoint as dj
dj.config["enable_python_native_blobs"] = True

import nwb_datajoint as nd
import ndx_franklab_novela.probe

data_dir = '/Users/loren/data/nwb_builder_test_data'
os.environ['NWB_DATAJOINT_BASE_DIR'] = data_dir
os.environ['KACHERY_STORAGE_DIR'] = os.path.join(data_dir, 'kachery-storage')


env: DJ_SUPPORT_FILEPATH_MANAGEMENT=TRUE


Please enter DataJoint username:  root
Please enter DataJoint password:  ········


Connecting root@localhost:3306


13:52:46 [I] klustakwik KlustaKwik2 version 0.2.6


#### Next we select the NWB file, which corresponds to the dataset we want to extract LFP from

In [3]:
nwb_file_names = nd.common.Nwbfile().fetch('nwb_file_name')
# take the first one for this demonstration
nwb_file_name = nwb_file_names[0]
print(nwb_file_name)

beans20190718.nwb


#### Create the standard LFP Filters. This only needs to be done once.

In [4]:
nd.common.FirFilter().create_standard_filters()

### Select all electrodes for LFP

In [5]:
electrode_ids = nd.common.Electrode.fetch('electrode_id')
nd.common.LFPSelection().set_lfp_electrodes(nwb_file_name, electrode_ids.tolist())


About to delete:
`common_ephys`.`l_f_p_selection__l_f_p_electrode`: 1 items
`common_ephys`.`l_f_p_selection`: 1 items


Proceed? [yes, No]:  no


Cancelled deletes.


### Or select one electrode for LFP


In [3]:
nd.common.LFPSelection().set_lfp_electrodes(nwb_file_name, [2])


About to delete:
`common_ephys`.`l_f_p_selection__l_f_p_electrode`: 1 items
`common_ephys`.`l_f_p_selection`: 1 items


Proceed? [yes, No]:  yes


Committed.


In [ ]:
### populate the LFP table. Note that this takes a few hours on a laptop if you use all electrodes

In [10]:
import time
tic = time.perf_counter()
nd.common.LFP().populate()
toc = time.perf_counter()

/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/datajoint/connection.py:218: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


writing new NWB file beans2019071800000000.nwb


ValueError: Cannot change container_source once set: 'root' pynwb.file.NWBFile

In [11]:
toc = time.perf_counter()

toc - tic

18972.756563538

In [8]:
nd.common.AnalysisNwbfile()

analysis_file_name the name of the file,nwb_file_name,analysis_file_description an optional description of this analysis,analysis_file_sha1 the sha1 hash of the NWB file for kachery,analysis_parameters additional relevant parmeters. Currently used only for analyses that span multiple NWB files


/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/datajoint/connection.py:218: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


nwb_file_name the name of the nwb file (same as nwb_file_location),linked_file_name the name of the linked file,linked_file_location the datajoint managed location of the linked file
